In [1]:
import pandas as pd
import json

data = pd.read_csv("../../assets/dataset/stumbleupon.tsv", sep='\t')
data['title'] = data.boilerplate.map(lambda x: json.loads(x).get('title', ''))
data['body'] = data.boilerplate.map(lambda x: json.loads(x).get('body', ''))
data.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title,body
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,24,0,5424,170,8,0.152941,0.079130,0,IBM Sees Holographic Calls Air Breathing Batte...,A sign stands outside the International Busine...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,40,0,4973,187,9,0.181818,0.125448,1,The Fully Electronic Futuristic Starting Gun T...,And that can be carried on a plane without the...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,55,0,2240,258,11,0.166667,0.057613,1,Fruits that Fight the Flu fruits that fight th...,Apples The most popular source of antioxidants...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,24,0,2737,120,5,0.041667,0.100858,1,10 Foolproof Tips for Better Sleep,There was a period in my life when I had a lot...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,14,0,12032,162,10,0.098765,0.082569,0,The 50 Coolest Jerseys You Didn t Know Existed...,Jersey sales is a curious business Whether you...


## Predicting "Greenness" Of Content

This dataset comes from [stumbleupon](https://www.stumbleupon.com/), a web page recommender.  

A description of the columns is below

FieldName|Type|Description
---------|----|-----------
url|string|Url of the webpage to be classified
title|string|Title of the article
body|string|Body text of article
urlid|integer| StumbleUpon's unique identifier for each url
boilerplate|json|Boilerplate text
alchemy_category|string|Alchemy category (per the publicly available Alchemy API found at www.alchemyapi.com)
alchemy_category_score|double|Alchemy category score (per the publicly available Alchemy API found at www.alchemyapi.com)
avglinksize| double|Average number of words in each link
commonlinkratio_1|double|# of links sharing at least 1 word with 1 other links / # of links
commonlinkratio_2|double|# of links sharing at least 1 word with 2 other links / # of links
commonlinkratio_3|double|# of links sharing at least 1 word with 3 other links / # of links
commonlinkratio_4|double|# of links sharing at least 1 word with 4 other links / # of links
compression_ratio|double|Compression achieved on this page via gzip (measure of redundancy)
embed_ratio|double|Count of number of <embed> usage
frameBased|integer (0 or 1)|A page is frame-based (1) if it has no body markup but have a frameset markup
frameTagRatio|double|Ratio of iframe markups over total number of markups
hasDomainLink|integer (0 or 1)|True (1) if it contains an <a> with an url with domain
html_ratio|double|Ratio of tags vs text in the page
image_ratio|double|Ratio of <img> tags vs text in the page
is_news|integer (0 or 1) | True (1) if StumbleUpon's news classifier determines that this webpage is news
lengthyLinkDomain| integer (0 or 1)|True (1) if at least 3 <a> 's text contains more than 30 alphanumeric characters
linkwordscore|double|Percentage of words on the page that are in hyperlink's text
news_front_page| integer (0 or 1)|True (1) if StumbleUpon's news classifier determines that this webpage is front-page news
non_markup_alphanum_characters|integer| Page's text's number of alphanumeric characters
numberOfLinks|integer Number of <a>|markups
numwords_in_url| double|Number of words in url
parametrizedLinkRatio|double|A link is parametrized if it's url contains parameters or has an attached onClick event
spelling_errors_ratio|double|Ratio of words not found in wiki (considered to be a spelling mistake)
label|integer (0 or 1)|User-determined label. Either evergreen (1) or non-evergreen (0); available for train.tsv only

> ### Let's try extracting some of the text content.
> ### Create a feature for the title containing 'recipe'. Is the % of evegreen websites higher or lower on pages that have recipe in the the title?

In [2]:
# Option 1: Create a function to check for this

def has_recipe(text_in):
    try:
        if 'recipe' in str(text_in).lower():
            return 1
        else:
            return 0
    except: 
        return 0
        
data['recipe'] = data['title'].map(has_recipe)

# Option 2: lambda functions

#data['recipe'] = data['title'].map(lambda t: 1 if 'recipe' in str(t).lower() else 0)


# Option 3: string functions
data['recipe'] = data['title'].str.contains('recipe')

In [3]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth= 3)

X = data[['image_ratio', 'html_ratio', 'recipe', 'label']].dropna()
y = X['label']
X.drop('label', axis=1, inplace=True)
    
    
# Fits the model
model.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [4]:
from sklearn.cross_validation import cross_val_score

# ... #

scores = cross_val_score(model, X, y, scoring='roc_auc', cv=5)
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.58417488  0.58045567  0.60179258  0.58135322  0.58409004], Average AUC 0.5863732768041588


C:\Users\Ayham\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 20, oob_score= True,max_depth= 3)
    
model.fit(X, y)

scores = cross_val_score(model, X, y, scoring='roc_auc', cv=5)
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

C:\Users\Ayham\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:439: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\Ayham\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:444: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


CV AUC [ 0.6107434   0.61097666  0.64544304  0.60393041  0.6214198 ], Average AUC 0.6185026611922189


C:\Users\Ayham\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:439: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\Ayham\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:444: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [100]:
from spacy.en import English
nlp_toolkit = English()

title = "IBM sees holographic calls, air breathing batteries"
parsed = nlp_toolkit(title)

for (i, word) in enumerate(parsed):
    print("Word: {}".format(word))
    print("\t Phrase type: {}".format(word.dep_))
    print("\t Is the word a known entity type? {}".format(word.ent_type_  if word.ent_type_ else "No"))
    print("\t Lemma: {}".format(word.lemma_))
    print("\t Parent of this word: {}".format(word.head.lemma_))


Word: IBM
	 Phrase type: nsubj
	 Is the word a known entity type? ORG
	 Lemma: ibm
	 Parent of this word: see
Word: sees
	 Phrase type: ROOT
	 Is the word a known entity type? No
	 Lemma: see
	 Parent of this word: see
Word: holographic
	 Phrase type: amod
	 Is the word a known entity type? No
	 Lemma: holographic
	 Parent of this word: call
Word: calls
	 Phrase type: dobj
	 Is the word a known entity type? No
	 Lemma: call
	 Parent of this word: see
Word: ,
	 Phrase type: punct
	 Is the word a known entity type? No
	 Lemma: ,
	 Parent of this word: call
Word: air
	 Phrase type: compound
	 Is the word a known entity type? No
	 Lemma: air
	 Parent of this word: breathing
Word: breathing
	 Phrase type: compound
	 Is the word a known entity type? No
	 Lemma: breathing
	 Parent of this word: battery
Word: batteries
	 Phrase type: appos
	 Is the word a known entity type? No
	 Lemma: battery
	 Parent of this word: call


In [101]:
parsed

IBM sees holographic calls, air breathing batteries

In [32]:
title = "Amazon THE has a high peak"
parsed = nlp_toolkit(title)

for (i, word) in enumerate(parsed):
    print("Word: {}".format(word))
    print("\t Phrase type: {}".format(word.dep_))
    print("\t Is the word a known entity type? {}".format(word.ent_type_  if word.ent_type_ else "No"))
    print("\t Lemma: {}".format(word.lemma_))
    print("\t Parent of this word: {}".format(word.head.lemma_))


Word: Amazon
	 Phrase type: compound
	 Is the word a known entity type? PERSON
	 Lemma: amazon
	 Parent of this word: the
Word: THE
	 Phrase type: nsubj
	 Is the word a known entity type? PERSON
	 Lemma: the
	 Parent of this word: have
Word: has
	 Phrase type: ROOT
	 Is the word a known entity type? No
	 Lemma: have
	 Parent of this word: have
Word: a
	 Phrase type: det
	 Is the word a known entity type? No
	 Lemma: a
	 Parent of this word: peak
Word: high
	 Phrase type: amod
	 Is the word a known entity type? No
	 Lemma: high
	 Parent of this word: peak
Word: peak
	 Phrase type: dobj
	 Is the word a known entity type? No
	 Lemma: peak
	 Parent of this word: have


## Investigate Page Titles

Let's see if we can find organizations in our page titles.

In [96]:
def references_organization(title):
    parsed = nlp_toolkit(title)
    return any([word.ent_type_ == 'ORG' for word in parsed])

data['references_organization'] = data['title'].fillna(u'').map(references_organization)

# Take a look
data[data['references_organization']][['title']].head()

,title
0,IBM Sees Holographic Calls Air Breathing Batte...
5,Genital Herpes Treatment
6,fashion lane American Wild Child
8,Valet The Handbook 31 Days 31 days
10,Business Financial News Breaking US Internatio...


## Exercise:

Lets write a function to identify titles that mention an organization (ORG) and a person (PERSON).

In [98]:
## Exercise solution
def references_org_person(title):
    parsed = nlp_toolkit(title)
    contains_org = any([word.ent_type_ == 'ORG' for word in parsed])
    contains_person = any([word.ent_type_ == 'PERSON' for word in parsed])
    return contains_org and contains_person

data['references_org_person'] = data['title'].fillna(u'').map(references_org_person)

# Take a look
data[data['references_org_person']][['title']].head()

,title
11,A Tip of the Cap to The Greatest Iron Man of T...
29,Genevieve Morton Swimsuit by Tyler Rose Swimwe...
44,Alyssa Miller Swimsuit by Charlie by Matthew Z...
114,Baby Gorilla Tries To Act Tough Video
115,BBC News UK Sweet message in a bottle


In [103]:

title = data.title.iloc[11]
parsed = nlp_toolkit(title)

def new_nlp(string):
    return nlp_toolkit(unicode(string))

for (i, word) in enumerate(parsed):
    print("Word: {}".format(word))
    print("\t Phrase type: {}".format(word.dep_))
    print("\t Is the word a known entity type? {}".format(word.ent_type_  if word.ent_type_ else "No"))
    print("\t Lemma: {}".format(word.lemma_))
    print("\t Parent of this word: {}".format(word.head.lemma_))


Word: A
	 Phrase type: det
	 Is the word a known entity type? No
	 Lemma: a
	 Parent of this word: tip
Word: Tip
	 Phrase type: ROOT
	 Is the word a known entity type? No
	 Lemma: tip
	 Parent of this word: tip
Word: of
	 Phrase type: prep
	 Is the word a known entity type? No
	 Lemma: of
	 Parent of this word: tip
Word: the
	 Phrase type: det
	 Is the word a known entity type? No
	 Lemma: the
	 Parent of this word: cap
Word: Cap
	 Phrase type: pobj
	 Is the word a known entity type? PERSON
	 Lemma: cap
	 Parent of this word: of
Word: to
	 Phrase type: prep
	 Is the word a known entity type? No
	 Lemma: to
	 Parent of this word: tip
Word: The
	 Phrase type: det
	 Is the word a known entity type? ORG
	 Lemma: the
	 Parent of this word: greatest
Word: Greatest
	 Phrase type: pobj
	 Is the word a known entity type? ORG
	 Lemma: greatest
	 Parent of this word: to
Word: Iron
	 Phrase type: compound
	 Is the word a known entity type? ORG
	 Lemma: iron
	 Parent of this word: man
Word: Man
	 P

 ### Demo: Use of the Count Vectorizer

In [108]:
titles = data['title'].fillna('')

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features = 1000, 
                             ngram_range=(1, 2), 
                             stop_words='english',
                             binary=False)

# Use `fit` to learn the vocabulary of the titles
vectorizer.fit(titles)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectorizer.transform(titles)

In [109]:
vectorizer.get_feature_names()

['000',
 '10',
 '10 best',
 '10 things',
 '10 ways',
 '100',
 '101',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '20',
 '20 best',
 '2007',
 '2008',
 '2008 sports',
 '2009',
 '2010',
 '2010 sports',
 '2011',
 '2011 sports',
 '2012',
 '2013',
 '2013 check',
 '2013 sports',
 '22',
 '24',
 '25',
 '30',
 '3d',
 '40',
 '50',
 '8211',
 '8217',
 'abs',
 'actually',
 'add',
 'advice',
 'aids',
 'air',
 'alcohol',
 'allrecipes',
 'allrecipes com',
 'almond',
 'alternative',
 'alyssa',
 'alyssa miller',
 'amazing',
 'america',
 'american',
 'anderson',
 'android',
 'angeles',
 'anniversary',
 'apple',
 'apples',
 'apps',
 'archive',
 'art',
 'arthritis',
 'asian',
 'athletes',
 'atlantic',
 'attack',
 'avocado',
 'awards',
 'awesome',
 'baby',
 'bacon',
 'bad',
 'bake',
 'baked',
 'baker',
 'bakers',
 'baking',
 'ball',
 'balls',
 'banana',
 'banana bread',
 'banned',
 'bar',
 'bars',
 'based',
 'basil',
 'bbc',
 'bbc food',
 'bbc news',
 'bbq',
 'beach',
 'bean',
 'bear',
 'beautif

In [110]:
X.shape

(7395, 1000)

In [114]:
vectorizer.get_feature_names()[43] in titles[0]

True

In [112]:
vectorizer.get_feature_names()[43]

'air'

In [113]:
titles[0]

'IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries'

In [78]:
print(X)

  (0, 43)	2
  (2, 210)	1
  (2, 351)	2
  (2, 360)	3
  (2, 383)	2
  (2, 437)	1
  (2, 566)	1
  (2, 567)	1
  (3, 1)	1
  (3, 102)	1
  (3, 803)	1
  (3, 908)	1
  (4, 35)	1
  (4, 237)	2
  (4, 507)	1
  (6, 53)	1
  (6, 340)	1
  (6, 973)	1
  (7, 217)	1
  (7, 270)	2
  (7, 476)	1
  (7, 477)	1
  (8, 268)	2
  (9, 139)	1
  (9, 231)	1
  :	:
  (7391, 213)	1
  (7392, 308)	1
  (7392, 309)	1
  (7392, 696)	1
  (7392, 867)	1
  (7394, 23)	2
  (7394, 24)	2
  (7394, 217)	1
  (7394, 311)	1
  (7394, 313)	1
  (7394, 393)	1
  (7394, 463)	2
  (7394, 464)	2
  (7394, 586)	1
  (7394, 587)	1
  (7394, 666)	1
  (7394, 667)	1
  (7394, 792)	2
  (7394, 793)	1
  (7394, 794)	1
  (7394, 832)	2
  (7394, 834)	2
  (7394, 869)	4
  (7394, 872)	1
  (7394, 873)	1


In [80]:
data['title'][0]

'IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries'

In [81]:
vectorizer.get_feature_names()[43]

'air'

 ### Demo: Build a random forest model to predict evergreeness of a website using the title features

In [84]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 20)
    
# Use `fit` to learn the vocabulary of the titles
vectorizer.fit(titles)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectorizer.transform(titles).toarray()
y = data['label']

from sklearn.cross_validation import cross_val_score

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.78984005  0.80565184  0.80523295], Average AUC 0.8002416143228364


### Exercise: Build a random forest model to predict evergreeness of a website using the title features and quantitative features

In [85]:
# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X_text_features = vectorizer.transform(titles)
from sklearn.cross_validation import cross_val_score

# Identify the features you want from the original dataset
other_features_columns = ['html_ratio', 'image_ratio']
other_features = data[other_features_columns]

# Stack them horizontally together
# This takes all of the word/n-gram columns and appends on two more columns for `html_ratio` and `image_ratio`
from scipy.sparse import hstack
X = hstack((X_text_features, other_features)).toarray()

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

# What features of these are most important?
model.fit(X, y)

all_feature_names = vectorizer.get_feature_names() + other_features_columns
feature_importances = pd.DataFrame({'Features' : all_feature_names, 'Importance Score': model.feature_importances_})
feature_importances.sort_values('Importance Score', ascending=False).head()

CV AUC [ 0.78345017  0.80135441  0.79386445], Average AUC 0.7928896790352962


,Features,Importance Score
1000,html_ratio,0.147923
1001,image_ratio,0.088108
719,recipe,0.038125
724,recipes,0.018735
184,chicken,0.013973


In [91]:
X[0:10,1000:]

array([[ 0.24583118,  0.00388349],
       [ 0.20348963,  0.08865248],
       [ 0.22640177,  0.12053571],
       [ 0.26565574,  0.03534304],
       [ 0.22888725,  0.05047319],
       [ 0.29829959,  0.03863636],
       [ 0.21897801,  0.31137725],
       [ 0.17374593,  0.02583026],
       [ 0.11496229,  1.13664596],
       [ 0.20511657,  0.20626151]])

 ### Exercise: Build a random forest model to predict evergreeness of a website using the body features

In [92]:
body_text = data['body'].fillna('')

# Use `fit` to learn the vocabulary
vectorizer.fit(body_text)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectorizer.transform(body_text).toarray()

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.83584387  0.86016107  0.83835443], Average AUC 0.8447864527251632


 ### Exercise: Use `TfIdfVectorizer` instead of `CountVectorizer` - is this an improvement?

In [95]:
titles = data['title'].fillna('')

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 1000, 
                             ngram_range=(1, 2), 
                             stop_words='english')


# Use `fit` to learn the vocabulary
vectorizer.fit(body_text)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectorizer.transform(body_text).toarray()

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.8441976   0.85565382  0.85009346], Average AUC 0.8499816273822391
